In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy as np
import tflearn
import tensorflow as tf
import random

Instructions for updating:
Colocations handled automatically by placer.


In [2]:
import json
with open('../intents/intents.json') as json_data:
    intents = json.load(json_data)

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/reynadoerwald/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
#classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

3113 documents
711 classes ['hypertensive disease', 'diabetes', 'depression mental', 'depressive disorder', 'coronary arteriosclerosis', 'coronary heart disease', 'pneumonia', 'failure heart congestive', 'accident cerebrovascular', 'asthma', 'myocardial infarction', 'hypercholesterolemia', 'infection', 'infection urinary tract', 'anemia', 'chronic obstructive airway disease', 'dementia', 'insufficiency renal', 'confusion', 'degenerative polyarthritis', 'hypothyroidism', 'anxiety state', 'malignant neoplasms', 'primary malignant neoplasm', 'acquired immuno-deficiency syndrome', 'HIV', 'hiv infections', 'cellulitis', 'gastroesophageal reflux disease', 'septicemia', 'systemic infection', 'sepsis (invertebrate)', 'deep vein thrombosis', 'dehydration', 'neoplasm', 'embolism pulmonary', 'epilepsy', 'cardiomyopathy', 'chronic kidney failure', 'carcinoma', 'hepatitis ', 'peripheral vascular disease', 'psychotic disorder', 'hyperlipidemia', 'bipolar disorder', 'obesity', 'ischemia', 'cirrhosis'

In [5]:
training = []
output = []

output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])

In [6]:
tf.reset_default_graph()

net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 389999  | total loss: 2.77924 | time: 0.911s
| Adam | epoch: 1000 | loss: 2.77924 - acc: 0.0808 -- iter: 3112/3113
Training Step: 390000  | total loss: 2.74532 | time: 0.914s
| Adam | epoch: 1000 | loss: 2.74532 - acc: 0.1102 -- iter: 3113/3113
--
INFO:tensorflow:/Users/reynadoerwald/Desktop/Honours/medical-chatbot/scripts/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [ ]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)

    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)

    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [8]:
p = bow("Nausea", words)
print (p)
print (classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [9]:
print(model.predict([p]))

[[2.16174126e-02 1.55766020e-02 7.35619915e-06 7.86964483e-06
  4.70084924e-05 5.12259976e-05 6.54667092e-05 5.86903298e-07
  7.83213109e-05 6.95861445e-06 5.94123208e-04 3.61351436e-03
  1.64364756e-03 5.91994067e-05 8.64345022e-03 4.22146620e-08
  3.30386544e-03 4.66482108e-03 1.57982548e-04 3.29718925e-03
  5.92789380e-03 2.59641733e-04 2.17310037e-03 1.98495435e-03
  7.47508238e-05 8.14281520e-05 8.36752879e-05 1.30512950e-03
  5.50307222e-02 2.35257430e-05 2.22284816e-05 2.22799918e-05
  1.56587979e-03 1.93243530e-02 4.20500385e-03 2.03523014e-04
  4.23450838e-05 2.24228628e-04 2.44551175e-03 9.55077261e-03
  8.77417642e-06 1.19723019e-03 2.49278492e-06 6.51820423e-03
  4.14678607e-06 1.74378720e-03 2.57474184e-03 9.50049981e-03
  1.96210039e-03 3.50732007e-05 6.90158573e-04 1.63574107e-02
  5.61038440e-04 1.92278294e-05 3.19314349e-05 4.86863498e-03
  2.29645681e-04 6.53051026e-03 7.52697986e-07 4.73901816e-02
  3.21702770e-04 3.45176013e-05 2.90473830e-02 2.70115817e-03
  2.6898

In [10]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [11]:
# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return print(random.choice(i['responses']))

            results.pop(0)

In [12]:
classify('Feeling nausea')

[('diverticulosis', 0.93682706)]

In [13]:
response('empysema pulmonary')

In [14]:
response('do you take cash?')

In [50]:
response('what kind of mopeds do you rent?')

In [51]:
response('Goodbye, see you later')

In [52]:
context

{}

In [53]:
response('we want to rent a moped')

We will be with you as soon as we can


In [54]:
context

{}

In [55]:
response('today')

In [56]:
classify('today')

[]

In [57]:
response("Hi there!", show_details=True)

In [58]:
response('today')
classify('today')

[]

In [59]:
response("thanks, your great")